In [64]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [65]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.input_block = nn.Sequential(
      #INPUT 28X28X1 >>CONV 3X3X1X16 >>26X26X16
          nn.Conv2d(1, 16, 3, bias=False), 
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.1),

 
      #INPUT 26X26X16 >>CONV 3X3X16X32 >>24X24X32
          nn.Conv2d(16, 32, 3, bias=False), 
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout2d(0.1),
      )
            # translation layer
      
      self.trans1 = nn.Sequential(
          #24X24x32 >>CONV 1X1X32X8 >>24X24X8
          nn.Conv2d(32, 8, 1, bias=False), 
          nn.ReLU(),
          #24X24x8 >>MAXPOOL (2,2) >>12X12X8
          nn.MaxPool2d(2, 2),
      )
      self.conv_block = nn.Sequential(
          #12X12x8 >>CONV 3X3X8X16 PAD=1 >>12X12X16
          nn.Conv2d(8, 16, 3,padding=1, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.1),

          #12X12x16 >>CONV 3X3X16X32 >>10X10X32

         nn.Conv2d(16, 32, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout2d(0.1),

      )
                  # translation layer
      self.trans2 = nn.Sequential(
          #10X10x32 >>CONV 1X1X32X8 >>10X10X8
          nn.Conv2d(32, 8, 1, bias=False), 
          nn.ReLU(),
          #10X10x8 >>MAXPOOL (2,2) >>5X5X8
          nn.MaxPool2d(2, 2),
      )
      self.conv_block2 = nn.Sequential(
        #5X5X8 >>CONV 3X3X8X16 PAD=1 >>5X5X16
         nn.Conv2d(8, 16, 3,padding=1,bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout2d(0.1),  
        #5X5X16 >>CONV 3X3X8X32 PAD=0 >>3X3X32
        nn.Conv2d(16, 32, 3,bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Dropout2d(0.1),

      )

          
      self.avg_pool = nn.Sequential(
      #3X3X32 >>CONV 1X1X32X10  >>3X3X10
      nn.Conv2d(32, 10, 1, bias=False),
      #3X3X10 >>AVG pool(3X3) >>1X1X10
      nn.AvgPool2d(3)
      )

    """forward: performs a forward pass when model(x) is called
    Params
        x: the input data
    Returns
        y: the output of the model
    """
    def forward(self, x):
        x = self.input_block(x)
        x = self.trans1(x)
        x = self.conv_block(x)
        x = self.trans2(x)
        x = self.conv_block2(x)
        x = self.avg_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [66]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             256
             ReLU-10            [-1, 8, 24, 24]               0
        MaxPool2d-11            [-1, 8, 12, 12]               0
           Conv2d-12           [-1, 16, 12, 12]           1,152
             ReLU-13           [-1, 16, 12, 12]               0
      BatchNorm2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [67]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [68]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        tqdm._instances.clear()


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [69]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print('\nEpoch {} : '.format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Epoch 1 : 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0583, Accuracy: 9830/10000 (98.30%)


Epoch 2 : 


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0178, Accuracy: 9939/10000 (99.39%)


Epoch 18 : 


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0167, Accuracy: 9949/10000 (99.49%)


Epoch 19 : 


loss=0.024139123037457466 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 31.40it/s]



Test set: Average loss: 0.0192, Accuracy: 9941/10000 (99.41%)

